In [19]:
import re
import spacy
from string import punctuation
from collections import Counter, OrderedDict
from spacy.symbols import ORTH, LEMMA, POS, TAG


nlp = spacy.load('pt_core_news_sm')

for n in nlp('Hoje é terça-feira'):
    print(n.text)

Hoje
é
terça
-
feira


### Criando casos especiais de tokens

#### Com regex pura

In [41]:
import re

PATTERN = r'''[\.,;:]+|\w+\-\w+|\w+'''

re.findall(PATTERN, 'Hoje é terça-feira. Segunda-feira fui ao médico e o Dr. Arnaldo em receitou um remédio.')

['Hoje',
 'é',
 'terça-feira',
 '.',
 'Segunda-feira',
 'fui',
 'ao',
 'médico',
 'e',
 'o',
 'Dr',
 '.',
 'Arnaldo',
 'em',
 'receitou',
 'um',
 'remédio',
 '.']

<br>
#### Criando casos especiais no Spacy

In [44]:
special_cases = {
    'segunda-feira': [{ORTH: u'segunda-feira', LEMMA: u'segunda', POS: u'NOUN'}],
    'terça-feira': [{ORTH: u'terça-feira', LEMMA: u'terça', POS: u'NOUN'}],
    'quarta-feira': [{ORTH: u'quarta-feira', LEMMA: u'quarta', POS: u'NOUN'}],
    'quinta-feira': [{ORTH: u'quinta-feira', LEMMA: u'quinta', POS: u'NOUN'}],
    'sexta-feira': [{ORTH: u'sexta-feira', LEMMA: u'sexta', POS: u'NOUN'}],
    'Segunda-feira': [{ORTH: u'Segunda-feira', LEMMA: u'Segunda', POS: u'NOUN'}],
    'Terça-feira': [{ORTH: u'Terça-feira', LEMMA: u'Terça', POS: u'NOUN'}],
    'Quarta-feira': [{ORTH: u'Quarta-feira', LEMMA: u'Quarta', POS: u'NOUN'}],
    'Quinta-feira': [{ORTH: u'Quinta-feira', LEMMA: u'Quinta', POS: u'NOUN'}],
    'Sexta-feira': [{ORTH: u'Sexta-feira', LEMMA: u'Sexta', POS: u'NOUN'}]
}

In [45]:
def add_cases(dicio_cases, nlp_obj):
    
    for key, value in dicio_cases.items():
        print(key, value)
        nlp_obj.tokenizer.add_special_case(key, value)
        
add_cases(special_cases, nlp)

segunda-feira [{65: 'segunda-feira', 73: 'segunda', 74: 'NOUN'}]
terça-feira [{65: 'terça-feira', 73: 'terça', 74: 'NOUN'}]
quarta-feira [{65: 'quarta-feira', 73: 'quarta', 74: 'NOUN'}]
quinta-feira [{65: 'quinta-feira', 73: 'quinta', 74: 'NOUN'}]
sexta-feira [{65: 'sexta-feira', 73: 'sexta', 74: 'NOUN'}]
Segunda-feira [{65: 'Segunda-feira', 73: 'Segunda', 74: 'NOUN'}]
Terça-feira [{65: 'Terça-feira', 73: 'Terça', 74: 'NOUN'}]
Quarta-feira [{65: 'Quarta-feira', 73: 'Quarta', 74: 'NOUN'}]
Quinta-feira [{65: 'Quinta-feira', 73: 'Quinta', 74: 'NOUN'}]
Sexta-feira [{65: 'Sexta-feira', 73: 'Sexta', 74: 'NOUN'}]


In [46]:
for n in nlp('Hoje é terça-feira. Segunda-feira fui ao médico e o Dr. Arnaldo em receitou um remédio.'):
    print(n.text)

Hoje
é
terça-feira
.
Segunda-feira
fui
a
o
médico
e
o
Dr.
Arnaldo
em
receitou
um
remédio
.


In [47]:
class TextNLP:
    
    def __init__(self, text):
        assert isinstance(text, str), "'A classe deve ser inicializada com uma string sendo passada por parâmetro'"
        
        if text == '':
            raise ValueError('A string passada não pode estar vazia')
        
        self.text = text
        self.doc = nlp(self.text)
        
        
    def __iter__(self):
        
        return (t.text for t in self.doc)
    
    
    def __repr__(self):
        tokens = self.words
        
        return "Texto: %s\nTokens: %s\nNum Tokens: %d" % (self.text, tokens, len(tokens))
    
    
    @property
    def words(self):
            
        return list(self.__iter__())
    
    
    def words_preprocessing(self, stop_words_remove=True, punctuation_remove=True):
        
        tokens = list(self.__iter__())
        
        if stop_words_remove:
            tokens = (tok for tok in tokens if tok not in stop_words_portuguese)
        
        elif punctuation_remove:
            tokens = (tok for tok in tokens if tok not in punctuation)
            
        else:
            tokens = (tok for tok in tokens)
            
        return tokens
    
    
    @property
    def sents(self):
                
        return (sent for sent in self.doc.sents)
    
    
    def tokens_counter(self, ordered=True, reverse=True):
        tok = list(self.__iter__())
        
        tok = OrderedDict(Counter(tok))
        
        if ordered:
            tok = OrderedDict(sorted(tok.items(), key=lambda x: x[1], reverse=reverse))
            
        return tok
        

In [48]:
a = TextNLP('Hoje é segunda-feira')

In [49]:
a

Texto: Hoje é segunda-feira
Tokens: ['Hoje', 'é', 'segunda-feira']
Num Tokens: 3